In this notebook, we present an example of how the way a system of ODE is written can affect the result of the smulation due to numerical error accumulation.

To exemplfy this, we will use the double pendulum system whose Hamiltonian is given by:


$$
    \mathcal{H}(\boldsymbol{\theta},\mathbf{p}) = \mathcal{T} + \mathcal{V} = \dfrac{1}{2} \mathbf{p}^T M^{-1}(\cos \Delta \theta) \, \mathbf{p} - \alpha(1+\sigma) \cos \theta_1 - \cos \theta_2
$$

where $\alpha = \dfrac{l_1}{l_2}$, $\sigma = \dfrac{m_1}{m_2}$, $\Delta \theta = \theta_1 - \theta_2$ and:

$$
    M^{-1}(x) = \dfrac{1}{1+\sigma - x^2}\begin{bmatrix}
        \dfrac{1}{\alpha^2} & -\dfrac{x}{\alpha} \\[.35cm]
        -\dfrac{x}{\alpha} & 1+\sigma
        \end{bmatrix}
$$

Then, Hamilton's equations of motion are:

$$
    \begin{cases}
        \dot{\boldsymbol{\theta}} = M^{-1}(\cos \Delta \theta) \, \mathbf{p} \\ 
        \dot{\mathbf{p}} = \dfrac{\sin \Delta \theta}{2} \mathbf{p}^T C(\cos \Delta \theta) \, \mathbf{p} \begin{bmatrix}
            1 \\
            -1
        \end{bmatrix} - \begin{bmatrix}
        \alpha(1+\sigma) \sin\theta_1 \\[.1cm]
        \sin\theta_2
        \end{bmatrix}
    \end{cases} 
$$

In [1]:
"""
Import libraries
"""
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp

In [2]:
"""
Define auxiliary functions
"""

"""
Define the potential energy function
"""
def V(theta1, theta2, a, s):
    return - a * (s + 1) * np.cos(theta1) - np.cos(theta2)

"""
Define the Hamiltonian function
"""
def H(theta1, theta2, p1, p2, a, s):

    delta = np.cos(theta1 - theta2)
    c = 1 + s - delta**2
    B_inv_matrix = np.array([
        [1 / (a**2 * c), -delta / (a * c)],
        [-delta / (a * c), (1 + s) / c]
    ])
    p = np.array([p1, p2])
    kinetic_energy = 0.5 * (p[0] * (B_inv_matrix[0, 0] * p[0] + B_inv_matrix[0, 1] * p[1]) +
                           p[1] * (B_inv_matrix[1, 0] * p[0] + B_inv_matrix[1, 1] * p[1]))
    potential_energy = V(theta1, theta2, a, s)
    return kinetic_energy + potential_energy

"""
Define the B_inv matrix function
"""
def B_inv_matrix(x, a, s):
    # x is an array of shape (N,) where N is the number of time steps
    N = len(x)
    # Precompute constants
    a2_1_plus_s = 1 / ((a**2) * (1 + s - x**2))
    one_plus_s = (1 + s) / (1 + s - x**2)
    # Calculate -a*cos(x) element-wise
    minus_a_cosx = - (x / (a * (1 + s - x**2)))

    # Create an empty array to hold N matrices, each of shape (2, 2)
    matrices = np.zeros((N, 2, 2))

    # Fill the matrices using broadcasting and element-wise assignment
    matrices[:, 0, 0] = a2_1_plus_s   # Broadcast scalar to all N matrices at [0, 0]
    matrices[:, 0, 1] = minus_a_cosx  # Assign the array to the [0, 1] element of each matrix
    matrices[:, 1, 0] = minus_a_cosx  # Assign the array to the [1, 0] element of each matrix
    matrices[:, 1, 1] = one_plus_s    # Broadcast scalar to all N matrices at [1, 1]

    return matrices

In [3]:
"""
Define the equivalent sets of Hamilton's equations for the double pendulum
"""

"""
System 1
"""
def double_pendulum_ode_1(t, Y, params):

    # --- Unpack parameters ---
    a, s = params # Expects params to be a tuple e.g., (a_value, s_value)

    # --- Get current state variables ---
    theta1 = Y[0]
    theta2 = Y[1]
    p1     = Y[2]
    p2     = Y[3]
    # LD_var = Y[4] # Only needed if its value affects derivatives

    # --- Calculate intermediate terms ---
    delta_theta = theta1 - theta2
    cos_delta = np.cos(delta_theta)
    sin_delta = np.sin(delta_theta)

    beta = a**2 * (1 + s)
    mu   = 1.0 / (1 + s)
    B1 = beta * (1 - mu * cos_delta * cos_delta)

    # --- Angle derivatves ---
    dtheta1dt = (p1 - a * p2 * cos_delta) / B1
    dtheta2dt = (beta * p2 - a * p1 * cos_delta) / B1

    denom = beta - a**2 * cos_delta * cos_delta
    C1 = sin_delta / (2 * denom * denom)

    # --- Calculate derivatives for momenta ---
    dp1dt = -2 * a * C1 * (beta * p2 - a * p1 * cos_delta) * (p1 - a * p2 * cos_delta) - (beta / a) * np.sin(theta1)
    dp2dt =  2 * a * C1 * (beta * p2 - a * p1 * cos_delta) * (p1 - a * p2 * cos_delta) - np.sin(theta2)
    

    # --- Calculate derivative for LD variable ---
    dLDdt = np.sqrt(abs(dtheta1dt)) + np.sqrt(abs(dtheta2dt)) + np.sqrt(abs(dp1dt)) + np.sqrt(abs(dp2dt))
    
    # dLDdt = math.sqrt(dtheta1dt**2 + dtheta2dt**2 + dp1dt**2 + dp2dt**2)

    # --- Create and return derivatives array ---
    # Create dYdt array inside the function
    dYdt = np.zeros_like(Y)
    dYdt[0] = dtheta1dt
    dYdt[1] = dtheta2dt
    dYdt[2] = dp1dt
    dYdt[3] = dp2dt
    dYdt[4] = dLDdt

    # Return the array
    return dYdt

"""
System 2
"""
def double_pendulum_ode_2(t, Y, params):
    # --- Unpack parameters ---
    a, s = params # Expects params to be a tuple e.g., (a_value, s_value)

    # --- Get current state variables ---
    theta1 = Y[0]
    theta2 = Y[1]
    p1     = Y[2]
    p2     = Y[3]
    
    delta = theta1 - theta2
    cos_delta = np.cos(delta)
    sin_delta = np.sin(delta)
    
    # Precompute a*(1+s)
    a_times_1_plus_s = a * (1 + s)
    beta = a * a_times_1_plus_s # beta = a*a*(1+s)
    mu   = 1.0 / (1 + s) # Keep mu for B1 calculation clarity, or use B1 = a*a*(1+s - cos_delta**2)
    
    # Compute the effective metric factor B1 once
    B1 = beta * (1 - mu * (cos_delta * cos_delta))
        
    # Compute derivatives for the angles first
    dtheta1dt = (p1 - a * p2 * cos_delta) / B1
    dtheta2dt = (beta * p2 - a * p1 * cos_delta) / B1
    
    # Compute simplified common term for momentum derivatives
    common_dp_term = a * sin_delta * dtheta1dt * dtheta2dt
    
    # Compute momentum derivatives using the simplified term and precomputed value
    dp1dt = -common_dp_term - a_times_1_plus_s * np.sin(theta1) # Used precomputed value
    dp2dt =  common_dp_term - np.sin(theta2)
    
    # Compute LD term directly using absolute values
    dLDdt = (np.sqrt(abs(dtheta1dt)) + np.sqrt(abs(dtheta2dt)) +
                np.sqrt(abs(dp1dt)) + np.sqrt(abs(dp2dt)))
    
    # dLDdt = math.sqrt(dtheta1dt**2 + dtheta2dt**2 + dp1dt**2 + dp2dt**2)
    
    # --- Create and return derivatives array ---
    # Create dYdt array inside the function
    dYdt = np.zeros_like(Y)
    dYdt[0] = dtheta1dt
    dYdt[1] = dtheta2dt
    dYdt[2] = dp1dt
    dYdt[3] = dp2dt
    dYdt[4] = dLDdt
    # Return the array
    return dYdt

In [ ]:
"""
Main function
"""
def main(flag):

    # Set system parameters:
    a = 1.0
    s = 1.0
    params = (a, s)

    if flag == 0:
        theta1 = 2.615195684873492077
        theta2 = 0.0
        p1 = -4.148704345580187081
        p2 = 4.287891959138516285
        LD0 = 0.0 
        Y0 = np.array([theta1, theta2, p1, p2, LD0])

    elif flag == 1:
        theta1 = 1.888458504855007103
        theta2 = 0.0
        p1 = 3.000865393046447060
        p2 = 3.462685225498120634
        LD0 = 0.0 
        Y0 = np.array([theta1, theta2, p1, p2, LD0])

    # Set integrtion parameters:
    t_start = 0.0
    t_final = 1000.0
    dt = 0.01
    num_points = int(round((t_final - t_start) / dt)) + 1
    t_eval = np.linspace(t_start, t_final, num_points)

    # Integrate system 1
    print("Integrating system 1...")
    sol_1 = solve_ivp(double_pendulum_ode_1, [t_start, t_final], Y0, args=(params,), method='BDF', t_eval=t_eval, rtol=1e-12, atol=1e-12)
    theta1_1 = sol_1.y[0, :]
    theta2_1 = sol_1.y[1, :]
    p1_1 = sol_1.y[2, :]
    p2_1 = sol_1.y[3, :]
    LD_1 = sol_1.y[4, :]
    print("System 1 integrated")

    # Integrate system 2
    print("Integrating system 2...")
    sol_2 = solve_ivp(double_pendulum_ode_2, [t_start, t_final], Y0, args=(params,), method='BDF', t_eval=t_eval, rtol=1e-12, atol=1e-12)
    theta1_2 = sol_2.y[0, :]
    theta2_2 = sol_2.y[1, :]
    p1_2 = sol_2.y[2, :]
    p2_2 = sol_2.y[3, :]
    LD_2 = sol_2.y[4, :]
    print("System 2 integrated")

    """
    Check energy conservation
    """
    H_1_init = H(theta1_1[0], theta2_1[0], p1_1[0], p2_1[0], a, s)
    H_2_init = H(theta1_2[0], theta2_2[0], p1_2[0], p2_2[0], a, s)
    H_1_final = H(theta1_1[-1], theta2_1[-1], p1_1[-1], p2_1[-1], a, s)
    H_2_final = H(theta1_2[-1], theta2_2[-1], p1_2[-1], p2_2[-1], a, s)
    print(f"Energy difference for system 1: {np.abs(H_1_final - H_1_init)}")
    print(f"Energy difference for system 2: {np.abs(H_2_final - H_2_init)}")

    """
    Calculate the condition number of matrix B_inv
    """
    B_1 = B_inv_matrix(theta1_1 - theta2_1, a, s)
    B_2 = B_inv_matrix(theta1_2 - theta2_2, a, s)
    cond_1 = np.linalg.cond(B_1)
    cond_2 = np.linalg.cond(B_2)

    """
    Plot the condition number as a function of time
    """
    plt.figure(figsize=(10, 8))
    plt.plot(t_eval, cond_1, label='System 1')
    plt.plot(t_eval, cond_2, label='System 2')
    plt.xlabel('Time')
    plt.ylabel('Condition number')
    plt.yscale('log')
    plt.legend()
    plt.show()

    """
    Plot the difference in the lagrangian descriptor as a function of time
    """
    plt.figure(figsize=(10, 8))
    plt.plot(t_eval, np.abs(LD_1 - LD_2), label='Difference in LD')
    plt.xlabel('Time')
    plt.ylabel('Difference in LD')
    plt.legend()
    plt.show()

    """
    Print the difference in the angle variables as a function of time
    """
    plt.figure(figsize=(10, 8))
    plt.plot(t_eval, np.abs(theta1_1 - theta1_2), label='Difference in theta1')
    plt.plot(t_eval, np.abs(theta2_1 - theta2_2), label='Difference in theta2')
    plt.xlabel('Time')
    plt.ylabel('Difference in theta')
    plt.legend()
    plt.show()

    """
    Print the difference in the momenta as a function of time
    """
    plt.figure(figsize=(10, 8))
    plt.plot(t_eval, np.abs(p1_1 - p1_2), label='Difference in p1')
    plt.plot(t_eval, np.abs(p2_1 - p2_2), label='Difference in p2')
    plt.xlabel('Time')
    plt.ylabel('Difference in momenta')
    plt.legend()
    plt.show()

    """
    Plot the euclidean distance between the two systems as a function of time
    """
    D = np.sqrt((theta1_1 - theta1_2)**2 + (theta2_1 - theta2_2)**2 + (p1_1 - p1_2)**2 + (p2_1 - p2_2)**2)
    plt.figure(figsize=(10, 8))
    plt.plot(t_eval, D, label='Euclidean distance')
    plt.xlabel('Time')
    plt.ylabel('Euclidean distance')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    flag = 1 # Set flag for the initial condition (o for regular, 1 for chaotic)
    main(flag)